In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd

In [3]:
koubei = pd.read_csv(
    'data/ijcai2016_koubei_train'
    , header=None, names=['user_id', 'merchant_id', 'location_id', 'timestamp']
#     , dtype=str
#     , parse_dates=[3]
#     , nrows=10
)

In [4]:
pred = pd.read_csv('data/ijcai2016_koubei_test', header=None, names=['user_id', 'location_id'])

# 生成测试数据

这里只取“预测集”里的和之前月份取交集的用户作为测试。因为给定的集合实际上由**线上、线下**两部分组成，因此，很难组合出和实际预测集相近的验证集。这样的话，虽然和实际情况并不相符，但是可以理解整个计算过程。

取 10 月份中“预测集”里有记录的用户。

In [5]:
koubei10 = pd.merge(
    koubei[koubei.timestamp.between(20151001, 20151030)][['user_id', 'location_id', 'merchant_id']].drop_duplicates(),
    pred[['user_id', 'location_id']].drop_duplicates(),
    on=['user_id', 'location_id'], how='inner'
).drop_duplicates()

In [6]:
len(koubei10)

28935

In [7]:
koubei10.head()

,user_id,location_id,merchant_id
0,357648,172,6540
1,1925368,172,6540
2,481455,172,8701
3,481455,172,820
4,1005895,172,437


取 11 月份中“预测集”里有记录的用户，作为实际的需要预测的集合。

In [8]:
koubei11 = pd.merge(
    koubei[koubei.timestamp.between(20151101, 20151131)][['user_id', 'location_id', 'merchant_id']].drop_duplicates(),
    pred[['user_id', 'location_id']].drop_duplicates(),
    on=['user_id', 'location_id'], how='inner'
).drop_duplicates()

In [9]:
koubei11.head()

,user_id,location_id,merchant_id
0,1765014,172,6540
1,1765014,172,263
2,1406251,172,6540
3,1406251,172,820
4,408834,172,6540


In [10]:
len(koubei11)

43294

假想的待预测集中商铺的流量

In [11]:
a=koubei11.groupby('merchant_id').size()

命中的“用户、地点、商家”集合，并聚合成商铺的流量

In [12]:
a.head()

merchant_id
1    2
2    1
3    5
4    3
7    7
dtype: int64

In [13]:
b=pd.merge(koubei10, koubei11, on=['user_id', 'location_id', 'merchant_id'], how='inner').groupby('merchant_id').size()

In [14]:
b.head()

merchant_id
4     3
10    4
12    5
14    1
16    3
dtype: int64

假想的预测集中商铺的流量

In [15]:
c=koubei10.groupby('merchant_id').size()

In [16]:
c.head()

merchant_id
2     2
3     1
4     4
8     2
10    5
dtype: int64

计算相关指标。

In [17]:
m = pd.DataFrame(dict(a=a, b=b))

In [18]:
m.head()

,a,b
merchant_id,,
1,2,NaN
2,1,NaN
3,5,NaN
4,3,3
7,7,NaN


In [19]:
count_nonzero(m.a.isnull())

0

In [20]:
count_nonzero(m.b.isnull())

1251

In [21]:
m['r'] = m.b / m.a

In [22]:
m.head()

,a,b,r
merchant_id,,,
1,2,NaN,NaN
2,1,NaN,NaN
3,5,NaN,NaN
4,3,3,1
7,7,NaN,NaN


In [23]:
len(m.r.dropna())

1298

In [24]:
count_nonzero(m.r == 1)

263

预测准确率

In [25]:
m.b.sum() / c.sum()

0.5264212890962502

预测召回率

In [26]:
m.b.sum() / a.sum()

0.35182704300826906